In [27]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [28]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [29]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [30]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (datetime.datetime.today(), json.dumps(neo)))
    conn.commit()

load_data()

Exploration of SQL transformation

In [31]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [32]:
df = pd.read_sql_query('''SELECT * FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2101955,2101955,101955 Bennu (1999 RQ36),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2101955,20.21,True,"[{'close_approach_date': '2023-08-25', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2101955?ap...,0.241299,0.539560,241.298697,539.560289,0.149936,0.335267,791.662417,1770.210979
1,3158115,3158115,(2003 QK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3158115,22.85,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3158115?ap...,0.071541,0.159971,71.540995,159.970528,0.044453,0.099401,234.714558,524.837708
2,3248972,3248972,(2004 OW10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3248972,24.61,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3248972?ap...,0.031809,0.071128,31.809363,71.127899,0.019765,0.044197,104.361432,233.359255
3,3408586,3408586,(2008 GS3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3408586,22.27,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3408586?ap...,0.093445,0.208949,93.444765,208.948847,0.058064,0.129835,306.577323,685.527735
4,3602525,3602525,(2012 GA5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3602525,24.19,False,"[{'close_approach_date': '2023-08-25', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3602525?ap...,0.038597,0.086306,38.597127,86.305799,0.023983,0.053628,126.630997,283.155518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,54169406,54169406,(2021 NQ5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54169406,21.20,True,"[{'close_approach_date': '2023-10-07', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54169406?a...,0.152952,0.342011,152.951935,342.010925,0.095040,0.212516,501.810828,1122.083122
169,54208043,54208043,(2021 TS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54208043,25.58,False,"[{'close_approach_date': '2023-10-07', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54208043?a...,0.020350,0.045503,20.349558,45.502995,0.012645,0.028274,66.763643,149.288045
170,54312823,54312823,(2022 TD),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312823,27.77,False,"[{'close_approach_date': '2023-10-07', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54312823?a...,0.007423,0.016597,7.422582,16.597397,0.004612,0.010313,24.352302,54.453404
171,54335221,54335221,(2022 YL),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54335221,26.54,False,"[{'close_approach_date': '2023-10-07', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54335221?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593


Extracting nested close approach data

In [33]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-08-25',
  'close_approach_date_full': '2023-Aug-25 06:51',
  'epoch_date_close_approach': 1692946260000,
  'relative_velocity': {'kilometers_per_second': '19.9688592017',
   'kilometers_per_hour': '71887.8931261816',
   'miles_per_hour': '44668.341148321'},
  'miss_distance': {'astronomical': '0.4967706275',
   'lunar': '193.2437740975',
   'kilometers': '74315827.752563425',
   'miles': '46177714.076388265'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [34]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}
C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2101955,2101955,101955 Bennu (1999 RQ36),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2101955,20.21,True,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2101955?ap...,0.241299,0.539560,241.298697,539.560289,0.149936,0.335267,791.662417,1770.210979
1,3158115,3158115,(2003 QK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3158115,22.85,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3158115?ap...,0.071541,0.159971,71.540995,159.970528,0.044453,0.099401,234.714558,524.837708
2,3248972,3248972,(2004 OW10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3248972,24.61,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3248972?ap...,0.031809,0.071128,31.809363,71.127899,0.019765,0.044197,104.361432,233.359255
3,3408586,3408586,(2008 GS3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3408586,22.27,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3408586?ap...,0.093445,0.208949,93.444765,208.948847,0.058064,0.129835,306.577323,685.527735
4,3602525,3602525,(2012 GA5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3602525,24.19,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3602525?ap...,0.038597,0.086306,38.597127,86.305799,0.023983,0.053628,126.630997,283.155518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,54169406,54169406,(2021 NQ5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54169406,21.20,True,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54169406?a...,0.152952,0.342011,152.951935,342.010925,0.095040,0.212516,501.810828,1122.083122
169,54208043,54208043,(2021 TS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54208043,25.58,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54208043?a...,0.020350,0.045503,20.349558,45.502995,0.012645,0.028274,66.763643,149.288045
170,54312823,54312823,(2022 TD),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312823,27.77,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312823?a...,0.007423,0.016597,7.422582,16.597397,0.004612,0.010313,24.352302,54.453404
171,54335221,54335221,(2022 YL),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54335221,26.54,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54335221?a...,0.013078,0.029244,13.078411,29.244216,0.008127,0.018172,42.908173,95.945593


In [35]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-08-25,2023-Aug-25 06:51,1692946260000,Earth,19.9688592017,71887.8931261816,44668.341148321,0.4967706275,193.2437740975,74315827.752563425,46177714.076388265
1,2023-08-25,2023-Aug-25 10:16,1692958560000,Earth,8.881183946,31972.2622056877,19866.3203688619,0.2937237596,114.2585424844,43940448.804552052,27303328.8149682376
2,2023-08-25,2023-Aug-25 21:29,1692998940000,Earth,4.177301952,15038.2870270498,9344.2067363355,0.2962462802,115.2398029978,44317812.513343174,27537811.7504928412
3,2023-08-25,2023-Aug-25 13:48,1692971280000,Earth,21.9081117957,78869.2024646521,49006.2552758834,0.4957086352,192.8306590928,74156955.966527024,46078995.7261759712
4,2023-08-25,2023-Aug-25 08:41,1692952860000,Earth,5.1097672483,18395.1620939486,11430.0383577649,0.1578803064,61.4154391896,23618557.552387368,14675891.1329389584
...,...,...,...,...,...,...,...,...,...,...,...
168,2023-10-07,2023-Oct-07 22:58,1696719480000,Earth,9.2647085264,33352.9506951528,20724.2265027742,0.1281802593,49.8621208677,19175493.767327691,11915099.3165204958
169,2023-10-07,2023-Oct-07 20:49,1696711740000,Earth,13.9558963399,50241.226823662,31217.944522736,0.3739510789,145.4669696921,55942284.887641943,34760923.9437190934
170,2023-10-07,2023-Oct-07 00:17,1696637820000,Earth,9.4008831737,33843.1794253792,21028.8355712863,0.0228353761,8.8829613029,3416123.625208907,2122680.7906886366
171,2023-10-07,2023-Oct-07 14:14,1696688040000,Earth,5.9601888903,21456.6800051366,13332.3465287472,0.2591173451,100.7966472439,38763402.907014937,24086461.6623506506


In [36]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,date_added,data
0,2101955,2101955,101955 Bennu (1999 RQ36),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2101955,20.21,True,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2101955?ap...,0.241299,...,Earth,19.9688592017,71887.8931261816,44668.341148321,0.4967706275,193.2437740975,74315827.752563425,46177714.076388265,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3158115,3158115,(2003 QK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3158115,22.85,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3158115?ap...,0.071541,...,Earth,8.881183946,31972.2622056877,19866.3203688619,0.2937237596,114.2585424844,43940448.804552052,27303328.8149682376,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3248972,3248972,(2004 OW10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3248972,24.61,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3248972?ap...,0.031809,...,Earth,4.177301952,15038.2870270498,9344.2067363355,0.2962462802,115.2398029978,44317812.513343174,27537811.7504928412,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3408586,3408586,(2008 GS3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3408586,22.27,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3408586?ap...,0.093445,...,Earth,21.9081117957,78869.2024646521,49006.2552758834,0.4957086352,192.8306590928,74156955.966527024,46078995.7261759712,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3602525,3602525,(2012 GA5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3602525,24.19,False,"{'close_approach_date': '2023-08-25', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3602525?ap...,0.038597,...,Earth,5.1097672483,18395.1620939486,11430.0383577649,0.1578803064,61.4154391896,23618557.552387368,14675891.1329389584,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,54169406,54169406,(2021 NQ5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54169406,21.20,True,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54169406?a...,0.152952,...,Earth,9.2647085264,33352.9506951528,20724.2265027742,0.1281802593,49.8621208677,19175493.767327691,11915099.3165204958,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
169,54208043,54208043,(2021 TS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54208043,25.58,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54208043?a...,0.020350,...,Earth,13.9558963399,50241.226823662,31217.944522736,0.3739510789,145.4669696921,55942284.887641943,34760923.9437190934,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
170,54312823,54312823,(2022 TD),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312823,27.77,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312823?a...,0.007423,...,Earth,9.4008831737,33843.1794253792,21028.8355712863,0.0228353761,8.8829613029,3416123.625208907,2122680.7906886366,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
171,54335221,54335221,(2022 YL),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54335221,26.54,False,"{'close_approach_date': '2023-10-07', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54335221?a...,0.013078,...,Earth,5.96

Cleaning up df

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 30 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    173 non-null    object 
 1   neo_reference_id                                      173 non-null    object 
 2   name                                                  173 non-null    object 
 3   nasa_jpl_url                                          173 non-null    object 
 4   absolute_magnitude_h                                  173 non-null    float64
 5   is_potentially_hazardous_asteroid                     173 non-null    bool   
 6   close_approach_data                                   173 non-null    object 
 7   is_sentry_object                                      173 non-null    bool   
 8   links.self                                            173 no

In [38]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,date_added,data
0,2101955,101955 Bennu (1999 RQ36),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2101955,20.21,True,False,241.298697,539.560289,2023-Aug-25 06:51,Earth,19.9688592017,71887.8931261816,0.4967706275,193.2437740975,74315827.752563425,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3158115,(2003 QK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3158115,22.85,False,False,71.540995,159.970528,2023-Aug-25 10:16,Earth,8.881183946,31972.2622056877,0.2937237596,114.2585424844,43940448.804552052,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3248972,(2004 OW10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3248972,24.61,False,False,31.809363,71.127899,2023-Aug-25 21:29,Earth,4.177301952,15038.2870270498,0.2962462802,115.2398029978,44317812.513343174,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3408586,(2008 GS3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3408586,22.27,False,False,93.444765,208.948847,2023-Aug-25 13:48,Earth,21.9081117957,78869.2024646521,0.4957086352,192.8306590928,74156955.966527024,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3602525,(2012 GA5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3602525,24.19,False,False,38.597127,86.305799,2023-Aug-25 08:41,Earth,5.1097672483,18395.1620939486,0.1578803064,61.4154391896,23618557.552387368,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,54169406,(2021 NQ5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54169406,21.20,True,False,152.951935,342.010925,2023-Oct-07 22:58,Earth,9.2647085264,33352.9506951528,0.1281802593,49.8621208677,19175493.767327691,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
169,54208043,(2021 TS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54208043,25.58,False,False,20.349558,45.502995,2023-Oct-07 20:49,Earth,13.9558963399,50241.226823662,0.3739510789,145.4669696921,55942284.887641943,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
170,54312823,(2022 TD),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312823,27.77,False,False,7.422582,16.597397,2023-Oct-07 00:17,Earth,9.4008831737,33843.1794253792,0.0228353761,8.8829613029,3416123.625208907,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
171,54335221,(2022 YL),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54335221,26.54,False,False,13.078411,29.244216,2023-Oct-07 14:14,Earth,5.9601888903,21456.6800051366,0.2591173451,100.7966472439,38763402.907014937,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [39]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
    'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
    'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
    'miss_distance.astronomical': 'miss_distance_astronomical',
    'miss_distance.lunar': 'miss_distance_lunar',
    'miss_distance.kilometers': 'miss_distance_kilometers',
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
0,2101955,101955 Bennu (1999 RQ36),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2101955,20.21,True,False,241.298697,539.560289,2023-Aug-25 06:51,Earth,19.9688592017,71887.8931261816,0.4967706275,193.2437740975,74315827.752563425,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3158115,(2003 QK5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3158115,22.85,False,False,71.540995,159.970528,2023-Aug-25 10:16,Earth,8.881183946,31972.2622056877,0.2937237596,114.2585424844,43940448.804552052,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3248972,(2004 OW10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3248972,24.61,False,False,31.809363,71.127899,2023-Aug-25 21:29,Earth,4.177301952,15038.2870270498,0.2962462802,115.2398029978,44317812.513343174,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3408586,(2008 GS3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3408586,22.27,False,False,93.444765,208.948847,2023-Aug-25 13:48,Earth,21.9081117957,78869.2024646521,0.4957086352,192.8306590928,74156955.966527024,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3602525,(2012 GA5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3602525,24.19,False,False,38.597127,86.305799,2023-Aug-25 08:41,Earth,5.1097672483,18395.1620939486,0.1578803064,61.4154391896,23618557.552387368,2023-10-08 09:24:20.108880,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,54169406,(2021 NQ5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54169406,21.20,True,False,152.951935,342.010925,2023-Oct-07 22:58,Earth,9.2647085264,33352.9506951528,0.1281802593,49.8621208677,19175493.767327691,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
169,54208043,(2021 TS10),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54208043,25.58,False,False,20.349558,45.502995,2023-Oct-07 20:49,Earth,13.9558963399,50241.226823662,0.3739510789,145.4669696921,55942284.887641943,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
170,54312823,(2022 TD),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312823,27.77,False,False,7.422582,16.597397,2023-Oct-07 00:17,Earth,9.4008831737,33843.1794253792,0.0228353761,8.8829613029,3416123.625208907,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...
171,54335221,(2022 YL),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54335221,26.54,False,False,13.078411,29.244216,2023-Oct-07 14:14,Earth,5.9601888903,21456.6800051366,0.2591173451,100.7966472439,38763402.907014937,2023-10-08 09:27:54.541830,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [40]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 10, 8)

In [41]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2023, 10, 8)

In [42]:
delta = (datetime.date.today() - last_execution_date).days
delta

0

In [43]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [44]:
delta = 8

for x in range (0, delta, 7):
    print(x)

0
7


In [45]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links() 
        

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-01&end_date=2023-10-07&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-24&end_date=2023-09-30&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [46]:
delta = 7
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    if delta % 7 == 0:
        # end_date = datetime.date.today()
        # start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        # end_date = end_date.strftime('%Y-%m-%d')
        # link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        # links.append(link)

        for x in range(0, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)  
        
    return links

get_links()

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-30&end_date=2023-10-07&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [47]:
delta = 15

for x in range (0, delta, 7):
    print(x)

def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (datetime.date.today() - datetime.timedelta(days=delta%7-1)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)


    for x in range(delta%7, delta, 7):
        end_date = (datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=x+6)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    return links

get_links() 

0
7
14


['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-08&end_date=2023-10-08&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-01&end_date=2023-10-07&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-24&end_date=2023-09-30&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [97]:
delta = 14
def get_links(delta):
    if delta == 0:
        return []

    links = []

    def build_link(x, y):
        end_date = x.strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=y)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta % 7 != 0:
        build_link(datetime.date.today(), delta%7-1)
    

    for x in range(delta%7, delta, 7):
        build_link((datetime.date.today() - datetime.timedelta(days=x)), x+6)

    return links

get_links(delta) 

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-02&end_date=2023-10-08&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-25&end_date=2023-10-01&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [105]:
def extract_load(links):
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''                   

            CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                               (datetime.datetime.today(), json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [77]:
def transform():
    df = pd.read_sql_query('''SELECT * FROM neo''', conn)

    df['data'] = df['data'].apply(json.loads)

    json_df = pd.json_normalize(df['data'])

    for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
                                             for k, v in element.items()}

    cad_df = pd.json_normalize(json_df['close_approach_data'])

    df = pd.concat([json_df, cad_df, df], axis=1)

    df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
                  'estimated_diameter.kilometers.estimated_diameter_min',
                  'estimated_diameter.kilometers.estimated_diameter_max',
                  'estimated_diameter.miles.estimated_diameter_min',
                  'estimated_diameter.miles.estimated_diameter_max',
                  'estimated_diameter.feet.estimated_diameter_min',
                  'estimated_diameter.feet.estimated_diameter_max',
                  'close_approach_date', 'epoch_date_close_approach',
                  'relative_velocity.miles_per_hour',
                  'miss_distance.miles'], axis=1
                 )

    df = df.rename(columns={
        'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
        'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
        'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
        'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
        'miss_distance.astronomical': 'miss_distance_astronomical',
        'miss_distance.lunar': 'miss_distance_lunar',
        'miss_distance.kilometers': 'miss_distance_kilometers',
        })

    cols = ['relative_velocity_kilometers_per_second',
            'relative_velocity_kilometers_per_hour',
            'miss_distance_astronomical',
            'miss_distance_lunar',
            'miss_distance_kilometers'
            ]

    df[cols] = df[cols].astype('float')
    
    df['close_approach_date_full'] = df['close_approach_date_full'].astype('datetime64[ns]')

    df['estimated_diameter_meters_mean'] = df[['estimated_diameter_meters_max',
                                               'estimated_diameter_meters_min']].mean(axis=1)

    return df

In [110]:
def drop_table():

    cursor = conn.cursor()

    cursor.executescript('''                   

        DROP TABLE IF EXISTS neo
                         
    ''')

    conn.commit()

In [111]:
drop_table()
extract_load(get_links(14))
df = transform()

df['close_approach_date_full'].sort_values().to_clipboard()

C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\800822385.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\800822385.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
C:\Users\Jack\AppData\Local\Temp\ipykernel_12720\800822385.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/p

In [87]:
df[df['close_approach_date_full'].duplicated()==True]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data,estimated_diameter_meters_mean


In [80]:
df[df['close_approach_date_full']=='2023-08-20 07:19:00']

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data,estimated_diameter_meters_mean


In [81]:
df_hazard = df[df['is_potentially_hazardous_asteroid']==1]
df_hazard_wow = df_hazard[df_hazard['miss_distance_lunar']<50]
df_hazard_wow

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data,estimated_diameter_meters_mean
151,54316689,(2022 UN),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54316689,21.59,True,False,127.807099,285.785361,2022-10-11 18:16:00,Earth,19.174668,69028.805604,0.058182,22.632884,8.703936e+06,2023-10-08 10:15:39.300016,{'links': {'self': 'http://api.nasa.gov/neo/re...,206.79623


In [82]:
df[df['estimated_diameter_meters_max']>1000]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data,estimated_diameter_meters_mean
23,2523823,523823 (2015 BG311),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2523823,18.33,False,False,573.528464,1282.448632,2022-10-15 22:51:00,Earth,8.326895,29976.823752,0.167915,65.319040,2.511977e+07,2023-10-08 10:15:39.292192,{'links': {'self': 'http://api.nasa.gov/neo/re...,927.988548
48,2011066,11066 Sigurd (1992 CC1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2011066,15.23,False,False,2390.864572,5346.135707,2022-10-12 16:00:00,Earth,23.618932,85028.153853,0.414127,161.095244,6.195246e+07,2023-10-08 10:15:39.294561,{'links': {'self': 'http://api.nasa.gov/neo/re...,3868.500140
72,2395289,395289 (2011 BJ2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2395289,18.23,False,False,600.558028,1342.888574,2022-10-13 05:55:00,Earth,34.364944,123713.798955,0.485175,188.732958,7.258110e+07,2023-10-08 10:15:39.294561,{'links': {'self': 'http://api.nasa.gov/neo/re...,971.723301
131,54322874,(2022 UY21),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54322874,18.76,False,False,470.494961,1052.058716,2022-10-10 00:20:00,Earth,19.880352,71569.267549,0.164864,64.132240,2.466336e+07,2023-10-08 10:15:39.294561,{'links': {'self': 'http://api.nasa.gov/neo/re...,761.276839


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       72 non-null     object        
 1   name                                     72 non-null     object        
 2   nasa_jpl_url                             72 non-null     object        
 3   absolute_magnitude_h                     72 non-null     float64       
 4   is_potentially_hazardous_asteroid        72 non-null     bool          
 5   is_sentry_object                         72 non-null     bool          
 6   estimated_diameter_meters_min            72 non-null     float64       
 7   estimated_diameter_meters_max            72 non-null     float64       
 8   close_approach_date_full                 72 non-null     datetime64[ns]
 9   orbiting_body                            72 n

In [83]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

data = df
source = ColumnDataSource(data)

p = figure()

p.circle(x='relative_velocity_kilometers_per_second', y='estimated_diameter_meters_max', source=source, fill_color='red')
p.circle(x='relative_velocity_kilometers_per_second', y='estimated_diameter_meters_min', source=source, fill_color='green')

show(p)

In [76]:
df['estimated_diameter_meters_mean'].sort_values(ascending=False).head()

55    1360.011613
52     834.723455
57     757.779089
29     691.102743
39     687.927415
Name: estimated_diameter_meters_mean, dtype: float64

In [74]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

data = df
source = ColumnDataSource(data)

p = figure()

p.circle(x='estimated_diameter_meters_mean', y='absolute_magnitude_h', source=source)

show(p)